In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [1]:
pwd

'/content'

In [3]:
ls


gdrive/  sample_data/


In [4]:
cd Data

[Errno 2] No such file or directory: 'Data'
/content


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

In [6]:
import os
import glob
import random

import numpy as np
from PIL import Image

In [7]:
import os
import glob
import random

import numpy as np
from PIL import Image

# Define the path to the data folder
data_folder = "/content/gdrive/MyDrive/Saad/Data"

# Define the image dimensions
input_height = 256
input_width = 256
input_channels = 1

# Load and preprocess the dataset
def load_dataset(folder):
    file_paths = glob.glob(os.path.join(folder, "*.png"))  # Assuming PNG format, modify if necessary
    dataset = []

    for file_path in file_paths:
        # Load and preprocess each image
        image = Image.open(file_path).convert("L")  # Load as grayscale image
        image = image.resize((input_width, input_height), Image.BICUBIC)  # Resize to desired dimensions
        image = np.array(image)  # Convert to numpy array
        image = np.expand_dims(image, axis=-1)  # Add channel dimension
        image = (image - 127.5) / 127.5  # Normalize to range [-1, 1]

        dataset.append(image)

    return np.array(dataset)





In [8]:
# Load the real depth maps and simulated depth maps
real_depth_dataset = load_dataset(os.path.join(data_folder, "Real_Depth"))
simulated_depth_dataset = load_dataset(os.path.join(data_folder, "Virtual/livingroom1-depth-clean/"))

# Shuffle the datasets
random.shuffle(real_depth_dataset)
random.shuffle(simulated_depth_dataset)

# Create TensorFlow Datasets
real_depth_dataset = tf.data.Dataset.from_tensor_slices(real_depth_dataset)
simulated_depth_dataset = tf.data.Dataset.from_tensor_slices(simulated_depth_dataset)

# Combine the datasets
batch_size=10
dataset = tf.data.Dataset.zip((real_depth_dataset, simulated_depth_dataset))
dataset = dataset.shuffle(buffer_size=1000).batch(batch_size)

In [9]:
def build_generator(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Downsampling
    x = layers.Conv2D(64, kernel_size=4, strides=2, padding='same')(inputs)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(128, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(256, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(512, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    # Upsampling
    x = layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Output
    x = layers.Conv2D(1, kernel_size=7, strides=1, padding='same', activation='tanh')(x)

    model = keras.models.Model(inputs, x)
    return model


def build_discriminator(input_shape):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, kernel_size=4, strides=2, padding='same')(inputs)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(128, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(256, kernel_size=4, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(512, kernel_size=4, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(1, kernel_size=4, strides=1, padding='same')(x)

    model = keras.models.Model(inputs, x)
    return model


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model


# Define the generator and discriminator models
#def build_generator(input_shape):
    # Implement the generator architecture
    # ...

#def build_discriminator(input_shape):
    # Implement the discriminator architecture
    # ...

# Build the generator and discriminator models
input_shape = (input_height, input_width, input_channels)
generator_A2B = build_generator(input_shape)
generator_B2A = build_generator(input_shape)
discriminator_A = build_discriminator(input_shape)
discriminator_B = build_discriminator(input_shape)

# Define the loss functions
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)

# Define the identity loss
def identity_loss(real_image, same_image):
    loss = tf.reduce_mean(tf.abs(real_image - same_image))
    return loss

# Define the cycle consistency loss
def cycle_loss(real_image, cycled_image):
    loss = tf.reduce_mean(tf.abs(real_image - cycled_image))
    return loss

# Define the adversarial loss
def adversarial_loss(real_image, generated_image):
    disc_generated_output = discriminator(generated_image)
    loss = loss_fn(tf.ones_like(disc_generated_output), disc_generated_output)
    return loss

# Define the generator and discriminator optimizers
generator_optimizer = Adam(learning_rate=2e-4, beta_1=0.5)
discriminator_optimizer = Adam(learning_rate=2e-4, beta_1=0.5)

# Define the training loop
@tf.function
def train_step(real_depth, real_simulated):
    with tf.GradientTape(persistent=True) as tape:
        # Generate simulated depth maps using real depth maps
        fake_simulated = generator_A2B(real_depth)
        # Generate real depth maps using simulated depth maps
        cycled_depth = generator_B2A(real_simulated)
        
        # Identity mapping
        same_depth = generator_B2A(real_depth)
        same_simulated = generator_A2B(real_simulated)
        
        # Compute the generator losses
        gen_A2B_loss = adversarial_loss(discriminator_B(fake_simulated))
        gen_B2A_loss = adversarial_loss(discriminator_A(cycled_depth))
        total_gen_loss = gen_A2B_loss + gen_B2A_loss + identity_loss(real_depth, same_depth) + identity_loss(real_simulated, same_simulated)
        
        # Compute the discriminator losses
        disc_A_loss = adversarial_loss(discriminator_A(real_depth), discriminator_A(fake_simulated))
        disc_B_loss = adversarial_loss(discriminator_B(real_simulated), discriminator_B(cycled_depth))
        
    # Calculate the gradients for generator and discriminator
    generator_gradients = tape.gradient(total_gen_loss, generator_A2B.trainable_variables + generator_B2A.trainable_variables)
    discriminator_gradients = tape.gradient(disc_A_loss, discriminator_A.trainable_variables) + tape.gradient(disc_B_loss, discriminator_B.trainable_variables)
    
    # Apply the gradients to the optimizer
    generator_optimizer.apply_gradients(zip(generator_gradients, generator_A2B.trainable_variables + generator_B2A.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator_A.trainable_variables + discriminator_B.trainable_variables))

# Start the training loop
epochs = 10
batch_size = 10

for epoch in range(epochs):
    # Iterate over the batches of the dataset
    for batch in dataset:
        real_depth, real_simulated = batch
        train_step(real_depth, real_simulated)
        
    # Print the training progress after each epoch
    print("Epoch {}/{}".format(epoch+1, epochs))




# Save the trained generator models
generator_A2B.save('generator_A2B.h5')
generator_B2A.save('generator_B2A.h5')# Example usage: Generate simulated depth maps from real depth maps
#fake_simulated = generator_A2B(real_depth)

# Example usage: Generate real depth maps from simulated depth maps
#cycled_depth = generator_B2A(real_simulated)


NameError: ignored

In [18]:
# Define the accuracy metric
accuracy = keras.metrics.BinaryAccuracy()

# Start the training loop
epochs = 100
batch_size = 16

for epoch in range(epochs):
    # Reset the accuracy metric at the beginning of each epoch
    accuracy.reset_states()

    # Iterate over the batches of the dataset
    for batch in dataset:
        real_depth, real_simulated = batch
        train_step(real_depth, real_simulated)

        # Calculate training accuracy
        fake_simulated = generator_A2B(real_depth)
        accuracy.update_state(tf.ones_like(fake_simulated), fake_simulated)

    # Compute training accuracy at the end of each epoch
    training_acc = accuracy.result()

    # Reset the accuracy metric for validation
    accuracy.reset_states()



    # Compute validation accuracy at the end of each epoch
    validation_acc = accuracy.result()

    # Print the training and validation accuracies after each epoch
    print("Epoch {}/{} - Training Accuracy: {:.4f} - Validation Accuracy: {:.4f}".format(epoch + 1, epochs, training_acc, validation_acc))


Epoch 1/100 - Training Accuracy: 0.0000 - Validation Accuracy: 0.0000
Epoch 2/100 - Training Accuracy: 0.0000 - Validation Accuracy: 0.0000
Epoch 3/100 - Training Accuracy: 0.0000 - Validation Accuracy: 0.0000
Epoch 4/100 - Training Accuracy: 0.0000 - Validation Accuracy: 0.0000
Epoch 5/100 - Training Accuracy: 0.0000 - Validation Accuracy: 0.0000
Epoch 6/100 - Training Accuracy: 0.0000 - Validation Accuracy: 0.0000
Epoch 7/100 - Training Accuracy: 0.0000 - Validation Accuracy: 0.0000
Epoch 8/100 - Training Accuracy: 0.0000 - Validation Accuracy: 0.0000
Epoch 9/100 - Training Accuracy: 0.0000 - Validation Accuracy: 0.0000
Epoch 10/100 - Training Accuracy: 0.0000 - Validation Accuracy: 0.0000
Epoch 11/100 - Training Accuracy: 0.0000 - Validation Accuracy: 0.0000
Epoch 12/100 - Training Accuracy: 0.0000 - Validation Accuracy: 0.0000
Epoch 13/100 - Training Accuracy: 0.0000 - Validation Accuracy: 0.0000
Epoch 14/100 - Training Accuracy: 0.0000 - Validation Accuracy: 0.0000
Epoch 15/100 - 